In [2]:
import gym
import matplotlib.pyplot as plt
import numpy as np
import random
import sys

from collections import defaultdict

%matplotlib inline

env = gym.make('Blackjack-v0')

In [3]:
def muestrea_politica(Q, estado,epsilon,i=1):
    if random.random() < epsilon/i: 
        accion = env.action_space.sample()
    else:
        if Q[estado,1]>Q[estado,0]: accion = 1
        else: accion = 0
            
    return accion

In [4]:
def inicializa_Q():
   estados = list()
   for mis_puntos in range(11,22):
        for sus_puntos in range(1,11):
            for utilizable in range(0,2):
                estados.append((mis_puntos,sus_puntos,utilizable))

   M = {}
   for estado in estados:
      mis_puntos, puntos_del_repartidor, as_utilizable = estado
      if (mis_puntos < 20):
         M[(estado,0)] = -0.001
         M[(estado,1)] = 0.001   # favorece pedir
      else:
         M[(estado,0)] = 0.001  # favorece quedarse
         M[(estado,1)] = -0.001
   return M

In [8]:
def prediccion_sarsa(politica, env, num_episodios, gama=1.0, alfa=1.0, eps=0.5):

    cuenta_retornos = defaultdict(float)

    ganados=0
    empatados=0
    
    Q = inicializa_Q()
    
    for i in range(0, num_episodios+1):
        S = env.reset()
        mis_puntos, _, as_utilizable = S 
        valor = mis_puntos + 10*as_utilizable

        while valor < 12:
            S, recompensa, termino, _ = env.step(1)
            mis_puntos, _, as_utilizable = S 
            valor = mis_puntos + 10*as_utilizable
        
        A = politica(Q,S,eps,(i+1)/1000000)
        termino=0

        while termino == 0:
            S_p, R, termino, _ = env.step(A)
            if termino:
              A_p = 0
              Q_p = R
            else:
              A_p = politica(Q,S_p,eps,(i+1)/1000000)
              Q_p = Q[S_p,A_p]
            Q[S,A] += alfa*( R + gama*Q_p - Q[S,A] )
            S=S_p
            A=A_p
        
        ganados += R>0
        empatados += R==0
        
    print("Episodios: " + str(i) +
          "\n   Ganados: " + str(ganados/i)+
          "\n   Empatados: " + str(empatados/i)+
          "\n   Perdidos: " + str(1-(empatados+ganados)/i))
    return Q

In [6]:
Q_10k = prediccion_sarsa(muestrea_politica, env, num_episodios=10000000, gama=0.9, alfa=0.5)

Núm. episodios: 10000000
   Ganados: 0.3847734
   Empatados: 0.0816835
   Perdidos: 0.5335430999999999


In [9]:
def imprime_politica(Q):
    print('---- Política ----')
    for useable in [0, 1]:
        if useable:
            print('As utilizable')
        else:
            print('As no utilizable')
        for val in range(21,10,-1):
            for card in range(1,11):
                if (Q[((val,card,useable),1)] > Q[((val,card,useable),0)]):
                    print('X',end="")
                else:
                    print(' ',end="")
            print('| %d' % val)
        print("A2345678910")
        print(' ')
    
imprime_politica(Q_10k)

---- Política ----
As no utilizable
          | 21
          | 20
          | 19
          | 18
X        X| 17
X    XXXXX| 16
XXX X XXXX| 15
XXXX XXXXX| 14
XXXXXX XXX| 13
XXXXXXXXXX| 12
XXXXXXXXXX| 11
A2345678910
 
As utilizable
          | 21
  XX     X| 20
XX XXXX XX| 19
X X XXXX X| 18
XXXXX XXXX| 17
XXXXXXXXXX| 16
XXXXXXXXXX| 15
XXXXXXXXXX| 14
XXXXXXXXX | 13
XXXXXXXXXX| 12
XXXXXXXXXX| 11
A2345678910
 
